# Lab 8 Regularization

## Exercise Questions:


Q1) Implement L2 regularization on cat-dog classification neural network. Train the model on the
dataset, and observe the impact of the regularization on the weight parameters. (Do not use data augmentation).

a. L2 regularization using optimizer’s weight decay

b. L2 regularization using loop to find L2 norm of weights

In [2]:
#trying to extract zipfile
import zipfile

zip_file = "data/cats_and_dogs_filtered.zip"
destination = "./data"

# Extract the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(destination)

print("Extraction completed.")

Extraction completed.


In [3]:
import torch
from torch import nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
import matplotlib.pyplot as plt
import glob

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
#set up transform function - to set data size and type to tensor
#Composes several transforms together
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [5]:
train_data = ImageFolder(root="data/cats_and_dogs_filtered/train", transform=transform)
test_data = ImageFolder(root="data/cats_and_dogs_filtered/validation", transform=transform)


train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=32)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
model = CNN().to(device)

/opt/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [9]:
def train_step(model, data_loader, loss_fn, optimizer, device):
    model.train()
    train_loss = 0
    for X, y in data_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        l2_reg = sum(torch.norm(param)**2 for param in model.parameters())
        loss += l2_reg * 0.01
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss /= len(data_loader)
    return train_loss


def test_step(model, data_loader, loss_fn, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()
            _, predicted = torch.max(y_pred, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    test_loss /= len(data_loader)
    accuracy = correct / total
    return test_loss, accuracy

In [12]:
torch.cuda.empty_cache()

In [13]:
num_epochs = 5
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}:")
    train_loss = train_step(model, train_loader, loss_fn, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}")
    test_loss, accuracy = test_step(model, test_loader, loss_fn, device)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1/5:
Train Loss: 1.4990
Test Loss: 0.6954, Accuracy: 0.5000
Epoch 2/5:
Train Loss: 1.0776
Test Loss: 0.6929, Accuracy: 0.5000
Epoch 3/5:
Train Loss: 0.9636
Test Loss: 0.6934, Accuracy: 0.5000
Epoch 4/5:
Train Loss: 0.8804
Test Loss: 0.6930, Accuracy: 0.5000
Epoch 5/5:
Train Loss: 0.9063
Test Loss: 0.6936, Accuracy: 0.5000


Q2) Implement L1 regularization on cat-dog classification neural network. Train the model on the
dataset, and observe the impact of the regularization on the weight parameters. (Do not use
data augmentation).

a. L1 regularization using optimizer’s weight decay

b. L1regularization using loop to find L1 norm of weight

In [ ]:
#same CNN model and data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

def train_step_l1(model, data_loader, loss_fn, optimizer, device, l1_lambda=0.01):
    model.train()
    train_loss = 0
    for X, y in data_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        l1_reg = sum(torch.norm(param, p=1) for param in model.parameters())
        loss += l1_reg * l1_lambda
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss /= len(data_loader)
    return train_loss

def test_step(model, data_loader, loss_fn, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()
            _, predicted = torch.max(y_pred, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    test_loss /= len(data_loader)
    accuracy = correct / total
    return test_loss, accuracy

num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}:")
    train_loss = train_step_l1(model, train_loader, loss_fn, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}")
    test_loss, accuracy = test_step(model, test_loader, loss_fn, device)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}")
